# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_csv = "../output_data/cities.csv"
weather_data_df = pd.read_csv(cities_csv)
weather_data_df.head(2)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Atuona,-9.80,-139.03,79.32,76,5,16.58,PF,1594850166
1,1,Saint-Augustin,51.23,-58.65,59.81,50,1,8.99,CA,1594850669


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# CONFIGURE GMAPS
gmaps.configure(api_key=g_key)

In [4]:
# LAT & LNG AS WEIGHT
locations = weather_data_df[['Lat','Lng']]
humidity = weather_data_df['Humidity']

#HEATMAP LAYER
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit ideal weather of less than 80 degrees but  more than 65, wind speeds of less than 10mph, and clear skies.

In [5]:
# NARROW ON TEMP
ideal_weather = weather_data_df.loc[(weather_data_df['Max Temp']<80) & (weather_data_df['Max Temp']>65)]
ideal_weather = ideal_weather.drop(columns = "Unnamed: 0")

#NARROW ON WIND SPEED
ideal_weather = ideal_weather.loc[(ideal_weather['Wind Speed']<10)]

#NARROW ON CLOUDYNESS
ideal_weather = ideal_weather.loc[(ideal_weather['Cloudiness']==0)]

print(f' There are {len(ideal_weather)} cities that fit my ideal temp preferences.')
ideal_weather.head(20)

 There are 17 cities that fit my ideal temp preferences.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,Alanya,36.54,32.00,75.20,94,0,2.24,TR,1594850675
70,Alcañiz,41.05,-0.13,72.00,49,0,1.99,ES,1594850682
195,Rudnyy,52.96,63.12,71.60,53,0,4.47,KZ,1594850707
283,Mahon,39.89,4.27,71.60,73,0,5.82,ES,1594850725
294,Petra,30.32,35.48,68.77,54,0,4.18,JO,1594850727
347,Zaysan,47.47,84.87,67.30,47,0,7.58,KZ,1594850737
352,Zliten,32.47,14.57,78.26,66,0,9.62,LY,1594850738
367,Qingdao,36.10,120.37,73.00,94,0,2.24,CN,1594850702
368,Shenyang,41.79,123.43,71.60,94,0,6.71,CN,1594850737
386,Abu Samrah,35.30,37.18,73.65,58,0,7.29,SY,1594850744


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
hotel_df = ideal_weather
hotel_df['Hotel Name'] = "" 

# RESET INDEX (cleaning for For-Loop)
hotel_df.index=range(len(hotel_df))
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Alanya,36.54,32.00,75.20,94,0,2.24,TR,1594850675,
1,Alcañiz,41.05,-0.13,72.00,49,0,1.99,ES,1594850682,
2,Rudnyy,52.96,63.12,71.60,53,0,4.47,KZ,1594850707,
3,Mahon,39.89,4.27,71.60,73,0,5.82,ES,1594850725,
4,Petra,30.32,35.48,68.77,54,0,4.18,JO,1594850727,
5,Zaysan,47.47,84.87,67.30,47,0,7.58,KZ,1594850737,
6,Zliten,32.47,14.57,78.26,66,0,9.62,LY,1594850738,
7,Qingdao,36.10,120.37,73.00,94,0,2.24,CN,1594850702,
8,Shenyang,41.79,123.43,71.60,94,0,6.71,CN,1594850737,
9,Abu Samrah,35.30,37.18,73.65,58,0,7.29,SY,1594850744,


In [41]:
# SET PARAMENTERS TO SEARCH FOR HOTELS WITHIN 5000 METERS
radius = 5000
target_type = 'lodging'

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for i in hotel_df.index:
    target_coordinates = hotel_df['Lat'][i].astype(str) + "," + hotel_df['Lng'][i].astype(str)

    params = {
      "location": target_coordinates,
      "radius": radius,
      "type": target_type,
      "key": g_key
    }

    response = requests.get(base_url, params=params)
    locations = response.json()

hotel_df['Hotel Name'][i] = locations["results"][0]["name"]
hotel_df.head(2)

# the following warning is OK

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Alanya,36.54,32.00,75.2,94,0,2.24,TR,1594850675,
1,Alcañiz,41.05,-0.13,72.0,49,0,1.99,ES,1594850682,


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_coordinates = hotel_df[["Lat", "Lng"]]
hotel_coordinates

,Lat,Lng
0,36.54,32.00
1,41.05,-0.13
2,52.96,63.12
3,39.89,4.27
4,30.32,35.48
5,47.47,84.87
6,32.47,14.57
7,36.10,120.37
8,41.79,123.43
9,35.30,37.18


In [48]:
# ADDING MARKER LAYER ON TOP OF MAP
humidity = hotel_df['Humidity']

fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(hotel_coordinates, weights=humidity, max_intensity = 100))
fig.add_layer(gmaps.marker_layer(hotel_coordinates, info_box_content=hotel_info))

# DISPLAY FIGURE
fig

Figure(layout=FigureLayout(height='420px'))